# 1. Загрузка и подготовка данных

In [2]:
import pandas as pd
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from numpy.random import RandomState
from scipy import stats as st
from math import sqrt

In [3]:
data_1 = pd.read_csv('/datasets/geo_data_0.csv')
data_2 = pd.read_csv('/datasets/geo_data_1.csv')
data_3 = pd.read_csv('/datasets/geo_data_2.csv')

In [4]:
print('data_1:')
print(data_1.info())
display(data_1.head())
print()
print('data_2:')
print(data_2.info())
display(data_2.head())
print()
print('data_3:')
print(data_3.info())
display(data_3.head())

data_1:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647



data_2:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305



data_3:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


Во всех трех датасетах данные полны, предобработка не требуется.

# 2. Обучение и проверка модели

Для решения задачи будем применять регрессию. Сравним модели случайного леса и линейной регрессии и выберем лучшую. для случайного леса будем подбирать гиперпараметры: глубину (max_depth) и количество деревьев (n_estimators).

In [5]:
%time

predictions_array = []
products_array = []
targets = []
models=[]

#разбиваем данные
i = 1
for data in [data_1, data_2, data_3]:
    features = data.drop(['id', 'product'], axis=1) #id не влияет на целевой показатель
    target = data['product']
    features_train, features_valid, target_train, target_valid = train_test_split(features,
                                                                                 target,
                                                                                 test_size=0.25,
                                                                                 random_state=12345)
#обучаем модель линейной регрессии
    model_linear = LinearRegression()
    model_linear.fit(features_train, target_train)
    linear_predictions = model_linear.predict(features_valid)
    score_linear = sqrt(mean_squared_error(target_valid, linear_predictions))
    print('Линейная регрессия, регион {}.'.format(i))
    print('Средний запас предсказанного сырья:', linear_predictions.mean())
    print('RMSE:', score_linear)
    print()
    #сохраняем предсказания и соответствующие им истинные значения
    predictions_array.append(linear_predictions)
    #products_array.append(target_valid)
    products_array.append(target_valid.reset_index(drop=True))
    targets.append(target_valid)
    models.append(model_linear)
        
    i += 1
    print()

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 6.44 µs
Линейная регрессия, регион 1.
Средний запас предсказанного сырья: 92.59256778438038
RMSE: 37.5794217150813


Линейная регрессия, регион 2.
Средний запас предсказанного сырья: 68.728546895446
RMSE: 0.893099286775616


Линейная регрессия, регион 3.
Средний запас предсказанного сырья: 94.96504596800489
RMSE: 40.02970873393434




**Вывод.** В 1 и 3 регионах модель предсказывает примерно одинаковое среднее количество сырья. Однако среднеквадратичная ошибка остается большой. Модель ошибается в среднем на 38,5 баррелей в этих регионах. Наибольшая точность - в регионе 2 (+/- один баррель). Однако и предсказанное среднее количество меньше (69 против 93 и 95).

# 3. Подготовка к расчёту прибыли

In [6]:
n_researched = 500 #необходимое для исследование кол-во точек
n_best = 200 #кол-во лучших точек из исследованных
budget = 10000000000 #бюджет на разработку скважин, руб.
income_per_unit = 450000 # доход на 1000 баррелей, руб.
max_risk = 0.025 #максимальный риск в %
drilling_cost = budget / 200 #стоимость бурения одной скважины, руб.

#минимально необходимое кол-во сырья для безубыточной разработки
min_fossil = int(drilling_cost / income_per_unit) + 1 #поскольку int отбрасывает дробную часть, добавляем 1 баррель
print('Минимальное количество сырья для безубыточной разработки:', min_fossil)

Минимальное количество сырья для безубыточной разработки: 112


**Вывод.** Требуемое для безубыточной работы количество сырья больше, чем средние предсказанные значения в любом из регионов. Однако в 1 и 3 регионах достаточно велик разброс предсказанных значений. Значит, там можно поискать скважины, где количество сырья будет больше минимально необходимого.

# 4. Расчёт прибыли и рисков 

In [7]:
#добавляем в данные столбцы с предсказаниями
i = 0
for data in [data_1, data_2, data_3]:
    features = data.drop(['id', 'product'], axis=1)
    data['predictions'] = models[i].predict(features)
    i+=1
    print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 6 columns):
id             100000 non-null object
f0             100000 non-null float64
f1             100000 non-null float64
f2             100000 non-null float64
product        100000 non-null float64
predictions    100000 non-null float64
dtypes: float64(5), object(1)
memory usage: 4.6+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 6 columns):
id             100000 non-null object
f0             100000 non-null float64
f1             100000 non-null float64
f2             100000 non-null float64
product        100000 non-null float64
predictions    100000 non-null float64
dtypes: float64(5), object(1)
memory usage: 4.6+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 6 columns):
id             100000 non-null object
f0             100000 non-null float64
f1             1

In [8]:
#функция расчета прибыли
def profit_calc(actual, predictions):
    pred_sorted = predictions.sort_values(ascending=False)
    actual_sorted = actual[pred_sorted.index][:200]
    revenue = round(actual_sorted.sum() * income_per_unit - drilling_cost * 200, 2)
    return revenue

#    prod = pd.Series(target).sort_values(ascending=False).head(200)
#    index = prod.index
#    return round((product.loc[index] * income_per_unit).sum() - drilling_cost * 200, 2)

In [9]:
#функция расчета прибыли
def profit_calc_1(target, product):
    pred_sorted = product.sort_values(ascending=False)
    #target=targets.reset_index(drop=True)
    selected = target[pred_sorted.index][:200]
    prod = selected.sum()
    revenue = prod * income_per_unit
    cost = drilling_cost * 200
    return revenue - cost

In [10]:
#расчет прибыли
i = 1
for data in [data_1, data_2, data_3]:
    actual = data['product']
    predictions = data['predictions']
    profit = profit_calc(actual, predictions)
    print('Расчетная прибыль в регионе {}:'.format(i), profit)
    i += 1

Расчетная прибыль в регионе 1: 3494104192.11
Расчетная прибыль в регионе 2: 2415086696.68
Расчетная прибыль в регионе 3: 2571410631.96


In [11]:
#бутстреп
state = RandomState(12345)
bootstrap_samples = 1000
profits=[]
profit_array=[]
region = 1
counts = []
count = 0

#изначальный вариант

#for data in [data_1, data_2, data_3]:
#    for i in range(bootstrap_samples):
#        #выборка 500 значений
#        predicted_sample = pd.Series(data['predictions'].sample(n=500, replace=True, random_state=state))
#        product_sample = data['product'][predicted_sample.index]
#        #расчет прибыли
#        profit = profit_calc(product_sample, predicted_sample)
#        profits.append(profit)

#второй вариант - бутстреп по предсказаниям

predictions_num = 1
for a in range(0, 3, 1): #счетчик для данных из трех регионов
    reg_values = pd.DataFrame({'Targets':targets[a], 'Predictions':predictions_array[a]})
    for i in range(bootstrap_samples):
        #выборка 500 знгачений
        predicted_sample = reg_values['Predictions'].sample(n=500, replace=True, random_state=state)
        product_sample = reg_values['Targets'][predicted_sample.index]     
        #расчет прибыли
        profit = profit_calc(product_sample, predicted_sample)
        profits.append(profit)
    
        #количетство случаев отрицательной прибыли
        if profit < 0:
            count +=1

    #сохраняем полученные выборки в массивах
    profit_array.append(profits)
    profits=[]
    counts.append(count)
    count=0
    a += 1

In [12]:
#средняя прибыль и доверительный интервал
region = 1
for a in range(0, 3):
    mean_profit = pd.Series(profit_array[a]).mean()
    profit_values = pd.Series(profit_array[a])
    confidence_interval = (round(profit_values.quantile(0.025), 2), round(profit_values.quantile(0.975), 2))
#            st.t.interval(0.95, 1, loc=mean_profit, scale=pd.Series(profit_array[a]).sem())
    risk = 1 * counts[a] / bootstrap_samples
    #risk = ((profit_values < 0).mean(), "%")
    print('Средняя прибыль в регионе {}:'.format(region), round(mean_profit, 2))
    print('Доверительный интервал для прибыли в регионе {}:'.format(region), confidence_interval)
    print('Вероятность убытков, %:', risk * 100)
    print()
    region += 1

Средняя прибыль в регионе 1: 425938526.91
Доверительный интервал для прибыли в регионе 1: (-102090094.84, 947976353.36)
Вероятность убытков, %: 6.0

Средняя прибыль в регионе 2: 518259493.7
Доверительный интервал для прибыли в регионе 2: (128123231.43, 953612982.06)
Вероятность убытков, %: 0.3

Средняя прибыль в регионе 3: 420194005.34
Доверительный интервал для прибыли в регионе 3: (-115852609.16, 989629939.85)
Вероятность убытков, %: 6.2



In [13]:
print(counts)

[60, 3, 62]


**Вывод.** Несмотря на то, что в третьем регионе модель предсказала наиболее богатые месторождения, техника бутстрепа показала, что в этом регионе наименьшая средняя прибыль и наибольший риск. С этой точки зрения наиболее привлекательным для разработки является второй регион. При риске всего в 2.1% потенциальная прибыль с 95% вероятностью составит от 0.2 до 1.2 млрд рублей. В сравнении с первым регионом риск выше всего на 0.1%, при этом прибыль выше.

На основе изложенного рекомендую к разработке второй регион.